In [97]:
import numpy as np
import pandas as pd
import cv2

In [98]:
img_path = 'janka_krvinky/PM1-1.tif'
csv_path = f'{img_path}.csv'
neighbor_dist = 150
size_per_pixel = 125.9 / 3264
df = pd.read_csv(csv_path, index_col=0)
df['Length long axis'] /= 2
df['Length short axis'] /= 2

df = df.rename(columns={'Length long axis': 'major_axis', 
                'Length short axis': 'minor_axis', 
                'X': 'x', 'Y': 'y', 
                'Rotation angle': 'rotation',
                'Aspect ratio': 'aspect_ratio',
                'Area': 'area'})
df

,Frame,Label,x,y,area,major_axis,minor_axis,aspect_ratio,rotation,type
,,,,,,,,,,
1,1,1645,17.361,15.937,4496.238,44.6070,32.0845,1.390,145.479,1
2,1,1646,1027.185,15.447,11968.927,112.4730,33.8735,3.320,3.292,1
3,1,1647,1140.647,48.877,47783.264,150.9970,100.7300,1.499,3.085,1
4,1,1648,491.351,60.341,44335.037,162.9180,86.6220,1.881,171.363,1
5,1,1650,659.365,85.922,44868.263,172.0965,82.9885,2.074,15.323,1
...,...,...,...,...,...,...,...,...,...,...
142,1,1822,1496.670,1132.607,11498.130,88.0025,41.5895,2.116,-8.717,0
143,1,1823,1205.113,1146.272,9236.661,76.0345,38.6680,1.966,24.354,0
144,1,1824,1040.658,1157.824,14035.871,90.4560,49.3915,1.831,4.076,0


In [99]:
import math
from enum import Enum
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2

def point_in_ellipse(ellipse_center, point, major_axis, minor_axis, angle):
    """
    Determine if a point is inside an ellipse.
    ellipse_center: (x, y) center of the ellipse
    point: (xp, yp) point to check
    major_axis: major axis of the ellipse
    minor_axis: minor axis of the ellipse
    angle: rotation angle of the ellipse in degres
    """
    angle = math.radians(angle)
    x, y = ellipse_center
    xp, yp = point
    cosa = math.cos(angle)
    sina = math.sin(angle)
    
    a_term = (cosa * (xp - x) + sina * (yp - y)) ** 2 / (major_axis / 2) ** 2
    b_term = (sina * (xp - x) - cosa * (yp - y)) ** 2 / (minor_axis / 2) ** 2
    
    return a_term + b_term <= 1

def boundary_points_ellipse(ellipse, num_points=100):
    """Generate boundary points for an ellipse."""
    h, k = ellipse['x'], ellipse['y']
    a = ellipse['major_axis'] / 2
    b = ellipse['minor_axis'] / 2
    phi = np.deg2rad(ellipse['rotation'])
    
    t_values = np.linspace(-np.pi, np.pi, num_points)
    
    return [
        (
            h + a * np.cos(t) * np.cos(phi) - b * np.sin(t) * np.sin(phi), 
            k + b * np.sin(t) * np.cos(phi) + a * np.cos(t) * np.sin(phi)
        ) 
        for t in t_values
    ]

class EllipseRel(Enum):
    FULLY_INSIDE = 1
    PARTIALLY_INSIDE = 2
    OUTSIDE = 3

def ellipse_relationship(ellipse_A, ellipse_B, num_points = 100) -> EllipseRel:
    """Determine the relationship between two ellipses. If A is inside B"""
    boundary_points_A = boundary_points_ellipse(ellipse_A, num_points)

    points_inside_count = 0

    for point in boundary_points_A:
        is_inside = point_in_ellipse((ellipse_B['x'], ellipse_B['y']), point, 
            ellipse_B['major_axis'], ellipse_B['minor_axis'], ellipse_B['rotation'])
        if is_inside:
            points_inside_count += 1
        elif not is_inside and points_inside_count >= 1:
            return EllipseRel.PARTIALLY_INSIDE
        
    if points_inside_count == len(boundary_points_A):
        return EllipseRel.FULLY_INSIDE
    elif points_inside_count > 0:
        return EllipseRel.PARTIALLY_INSIDE
    else:
        return EllipseRel.OUTSIDE

def ellipse_to_box(ellipse):
    """Convert an ellipse to a bounding box."""
    x, y = ellipse['x'], ellipse['y']
    major_axis, minor_axis = ellipse['major_axis'], ellipse['minor_axis']
    phi = math.radians(ellipse['rotation'])

    ux = major_axis / 2 * math.cos(phi)
    uy = major_axis / 2 * math.sin(phi)
    vx = minor_axis / 2 * math.cos(phi + math.pi/2)
    vy = minor_axis / 2 * math.sin(phi + math.pi/2)

    bbox_halfwidth = math.sqrt(ux*ux + vx*vx)
    bbox_halfheight = math.sqrt(uy*uy + vy*vy)

    return {
        'y_max': y + bbox_halfheight,
        'y_min': y - bbox_halfheight,
        'x_max': x + bbox_halfwidth,
        'x_min': x - bbox_halfwidth
    }

def get_neighbors(point, df, tree, radius=150):
    neighbors = tree.query_radius(point, r=radius)[0]
    return df[df.index.isin(neighbors + 1)]

In [100]:
def filter_bodies_with_multiple_or_zero_cores(df):
    bodies_df = df[df['type'] == 1]
    cores_df = df[df['type'] == 0]

    # Initialize a list to store valid body indices
    valid_body_indices = []

    # Check each body for the number of cores inside
    for body_id, body in bodies_df.iterrows():        
        cores_inside_count = 0
        for core_id, core in cores_df.iterrows():            
            if point_in_ellipse((body['x'], body['y']), (core['x'], core['y']), body['major_axis'], body['minor_axis'], body['rotation']):
                cores_inside_count += 1
            
            # If more than one core is inside, break the loop
            if cores_inside_count > 1:
                break
        
        # If only one core is inside, add the body index to the valid list
        if cores_inside_count == 1:
            valid_body_indices.append(body_id)

    # Filter out the valid bodies from the dataframe
    return df[df.index.isin(valid_body_indices) | (df['type'] == 0)]

In [101]:
def filter_overlaping_cores(df, ntree, radius=150):
    cores_df = df[df['type'] == 0]

    # Initialize a list to store valid body indices
    valid_core_indices = set(cores_df.index)

    # Check every core pair if they overlap
    for core_id_A, core_A in cores_df.iterrows():
        if core_id_A not in valid_core_indices:
            continue

        neighbors = get_neighbors([core_A[['x', 'y']].tolist()], cores_df, ntree, radius=radius)
        for core_id_B, core_B in neighbors.iterrows():
            if core_id_A == core_id_B:
                continue
            if core_id_B not in valid_core_indices:
                continue
            
            el_rel = ellipse_relationship(core_A, core_B, 32)
            if el_rel != EllipseRel.OUTSIDE:
                valid_core_indices.remove(core_id_A)
                valid_core_indices.remove(core_id_B)
                break
    
    # Filter out the valid bodies from the dataframe
    return df[df.index.isin(valid_core_indices) | (df['type'] == 1)]

In [102]:
def filter_ellipses_on_border(df, x_min=0, x_max=1632, y_min=0, y_max=1224):
    ellipses_inside = set()

    for id, row in df.iterrows():
        bbox = ellipse_to_box(row)
        if bbox['x_min'] > x_min and bbox['x_max'] < x_max and bbox['y_min'] > y_min and bbox['y_max'] < y_max:
            ellipses_inside.add(id)

    return df[df.index.isin(ellipses_inside)]

In [103]:
def join_cores_with_bodies(df, tree):
    bodies_df = df[df['type'] == 1]
    cores_df = df[df['type'] == 0]

    body_core_pair = {}

    # Check each body for the number of cores inside
    for body_id, body in bodies_df.iterrows():
        core_neighbors = get_neighbors([body[['x', 'y']].tolist()], cores_df, tree, radius=100)
        for core_id, core in core_neighbors.iterrows():
            
            if ellipse_relationship(core, body, 24) == EllipseRel.FULLY_INSIDE:
                body_core_pair[body_id] = core_id
                break

    return body_core_pair

In [104]:
def filter_unassigned_cores(df, body_core_map):
    has_body = set(body_core_map.values())
    bodies = set(body_core_map.keys())

    return df[df.index.isin(has_body) | df.index.isin(bodies)]

In [105]:
def ellipse_perimeter(ellipse):
    a = ellipse['major_axis'] / 2
    b = ellipse['minor_axis'] / 2
    h = (a - b) ** 2 / (a + b) ** 2

    return math.pi * (a + b) * (1 + (3 * h / (10 + math.sqrt(4 - 3 * h))))

In [106]:
def measure_blood_cell(df, body_core_map, size_per_pixel):
    df = df.copy()
    idx, body_major_length, body_minor_length, body_perimeter = [], [], [], []
    core_major_length, core_minor_length, core_perimeter = [], [], []

    df['major_axis'] *= size_per_pixel * 2
    df['minor_axis'] *= size_per_pixel * 2
    df_bodies = df[df['type'] == 1]


    for id, row in df_bodies.iterrows():
        if id not in body_core_map:
            continue

        idx.append(id)
        body_major_length.append(row['major_axis'])
        body_minor_length.append(row['minor_axis'])
        body_perimeter.append(ellipse_perimeter(row))

        core = df.loc[body_core_map[id]]

        core_major_length.append(core['major_axis'])
        core_minor_length.append(core['minor_axis'])
        core_perimeter.append(ellipse_perimeter(core))

    return pd.DataFrame({
        'id': idx,
        'body_major_length': body_major_length,
        'body_minor_length': body_minor_length,
        'body_perimeter': body_perimeter,
        'core_major_length': core_major_length,
        'core_minor_length': core_minor_length,
        'core_perimeter': core_perimeter
    })

In [149]:
from matplotlib.patches import Circle
from sklearn.neighbors import KDTree


def show_neighbors(radius=150):
    tree = KDTree(df[['x', 'y']])

    # Select a random row from df
    sampled = df.loc[[239]]

    # Get neighbors within a radius of 500
    neighbors = tree.query_radius(sampled[['x', 'y']], r=radius)[0]
    fig, ax = plt.subplots(figsize=(12, 12))
        
    img = cv2.imread(img_path)
    img = cv2.resize(img, (1632, 1224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)

    circle = Circle((sampled['x'], sampled['y']), radius, fill=False)
    ax.add_patch(circle)

    neighbors_df = df.loc[neighbors+1]
    for index, row in neighbors_df.iterrows():
        if index == sampled.index:
            continue

        ellipse_color = 'blue' if row['type'] == 1 else 'yellow'
        ellipse_patch = patches.Ellipse(
            (row['x'], row['y']),
            width=row['major_axis'],
            height=row['minor_axis'],
            angle=row['rotation'],
            color=ellipse_color,
            alpha=0.3
        )
        ax.add_patch(ellipse_patch)
        ax.text(row['x'], row['y'], index, color='black', fontsize=12)

    # Extract and plot ellipses for the body (type = 1) and core (type = 0)
    ellipse_color = 'green'
    ellipse_patch = patches.Ellipse(
        (sampled['x'], sampled['y']),
        width=sampled['major_axis'],
        height=sampled['minor_axis'],
        angle=sampled['rotation'],
        color=ellipse_color,
        alpha=0.3
    )
    ax.add_patch(ellipse_patch)

    # Set plot limits and aspect ratio
    ax.set_xlim(0, df['x'].max())
    ax.set_ylim(0, df['y'].max())
    ax.set_aspect('equal', 'box')
    ax.invert_yaxis()
    

    plt.show()

# show_neighbors(neighbor_dist)

In [108]:
%matplotlib qt


In [160]:
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from matplotlib.backend_bases import MouseButton


def draw_image(img_path, df, body_core_map, title="Red Blood Cells - Body (Blue) and Core (Red)", show_body_ids=False, measurements = None):
    fig, ax = plt.subplots(figsize=(12, 12))

    img = cv2.imread(img_path)
    height, width, _ = img.shape
    width, height = width // 2, height // 2
    img = cv2.resize(img, (width, height))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    ax.imshow(img)

    delete_body_ids = []
    delete_ellipses = []
    ellipses = {}

    def on_press(event):
        if event.key == 'z' and len(delete_ellipses) > 0:
            ellipse_core = delete_ellipses.pop()
            ellipse_body = delete_ellipses.pop()
            delete_body_ids.remove(ellipse_body.id)
            ellipse_body.set_visible(True)
            ellipse_core.set_visible(True)
            fig.canvas.draw()

    def onpick(event):
        if isinstance(event.artist, Ellipse):
            ellipse_body = event.artist
            ellipse_core = ellipses[body_core_map[ellipse_body.id]]
            delete_body_ids.append(ellipse_body.id)
            delete_ellipses.append(ellipse_body)
            delete_ellipses.append(ellipse_core)
            ellipse_body.set_visible(False)
            ellipse_core.set_visible(False)
            fig.canvas.draw()

    # Extract and plot ellipses for the body (type = 1) and core (type = 0)
    for index, row in df.iterrows():
        ellipse_color = 'blue' if row['type'] == 1 else 'yellow'
        ellipse_patch = Ellipse(
            (row['x'], row['y']),
            width=row['major_axis'],
            height=row['minor_axis'],
            angle=row['rotation'],
            color=ellipse_color,
            alpha=0.3,
            picker=True if row['type'] == 1 else None,
        )
        ellipse_patch.id = index
        ellipses[index] = ellipse_patch
        ax.add_patch(ellipse_patch)
        if show_body_ids and row['type'] == 1:
            text = index
            if measurements is not None:
                row_measurements = measurements.loc[measurements['id'] == index]
                text = f"{index} - {row_measurements['body_major_length'].item():.2f}x{row_measurements['body_minor_length'].item():.2f}"
            ax.text(row['x'], row['y'], text, color='black', fontsize=10)

    # Set plot limits and aspect ratio
    ax.set_xlim(0 - 50, width + 50)
    ax.set_ylim(0 - 50, height + 50)
    ax.set_aspect('equal', 'box')
    ax.invert_yaxis()

    fig.canvas.mpl_connect('key_press_event', on_press)
    fig.canvas.mpl_connect('pick_event', onpick)
    
    plt.title(title)
    plt.show()

    return delete_body_ids

In [118]:
import cv2
from sklearn.neighbors import KDTree
import time

tree = KDTree(df[['x', 'y']])

start = time.time()
df_filtered = filter_ellipses_on_border(df)
df_filtered = filter_overlaping_cores(df_filtered, tree)
df_filtered = filter_bodies_with_multiple_or_zero_cores(df_filtered)
body_core_map = join_cores_with_bodies(df_filtered, tree)
df_filtered = filter_unassigned_cores(df_filtered, body_core_map)
end = time.time()
print('Filter took', end - start, 's')

# draw_image(img_path, df)
to_delete = draw_image(img_path, df_filtered, body_core_map)
to_delete = to_delete + [body_core_map[k] for k in to_delete]
df_filtered = df_filtered[~df_filtered.index.isin(to_delete)]

df_measured = measure_blood_cell(df_filtered, body_core_map,  size_per_pixel)
df_measured.to_csv(f'{img_path}.measured.csv')

Filter took 0.5351276397705078 s


In [132]:
df_filtered

,Frame,Label,x,y,area,major_axis,minor_axis,aspect_ratio,rotation,type
,,,,,,,,,,
4,1,1648,491.351,60.341,44335.037,162.9180,86.6220,1.881,171.363,1
5,1,1650,659.365,85.922,44868.263,172.0965,82.9885,2.074,15.323,1
6,1,1651,860.983,93.089,28957.963,112.6540,81.8220,1.377,168.719,1
7,1,1652,1467.304,118.209,46751.311,145.1180,102.5470,1.415,26.209,1
8,1,1654,375.961,113.929,46259.068,147.4410,99.8685,1.476,12.321,1
...,...,...,...,...,...,...,...,...,...,...
138,1,1816,416.861,1062.811,9307.708,74.4560,39.7915,1.871,-36.266,0
140,1,1818,804.156,1079.191,8697.146,67.8675,40.7910,1.664,27.412,0
141,1,1821,671.419,1118.024,8733.004,69.6615,39.9045,1.746,13.975,0


In [161]:
draw_image(img_path, df_filtered, body_core_map)

[]